In [1]:
# основные библиотеки
import pandas as pd
import numpy as np
import json

# семантический анализ
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from afinn import Afinn
from textblob import TextBlob, Blobber
from textblob.sentiments import NaiveBayesAnalyzer
import re
import string

# визуализация
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRanker
from sklearn import tree
from sklearn import ensemble

In [2]:
# функция загрузки датасета

def load(path):
    comment_data = pd.read_json(path, lines=True, orient="records", encoding="windows-1251")
    comment_data.rename({"text":"post"}, inplace=True, axis=1)
    comment_data.set_index("post", inplace=True)
    comment_data = comment_data.explode(column="comments")
    comments = pd.json_normalize(comment_data["comments"])
    comments.set_index(comment_data.index, inplace=True)
    comment_data = pd.concat([comment_data, comments], axis=1)
    comment_data.drop("comments", axis=1, inplace=True)
    comment_data.reset_index(inplace=True)
    comment_data['text'] = comment_data['text'].str.replace("&#x27;", "'")
    del comments
    return comment_data

In [3]:
train_df = load("files/ranking_train.jsonl")
test_df = load("files/ranking_test.jsonl")

In [4]:
train_df

,post,text,score
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
...,...,...,...
440530,Pay your rent with a Credit or Debit card. No ...,Most major banks offer a service called 'bill ...,0
440531,Pay your rent with a Credit or Debit card. No ...,"It costs 3.25%, or $74.25 for the example of $...",1
440532,Pay your rent with a Credit or Debit card. No ...,As many other comments have pointed out almost...,2
440533,Pay your rent with a Credit or Debit card. No ...,My apartment building uses Yapstone's RentPaym...,3


# Sentiment Intensity Analyzer

Сделаем анализ для текста комментариев.

In [5]:
sia_analysis = SentimentIntensityAnalyzer()

def sia_text_analysis(df):
    """функция для получения различных коэффициентов семантической окраски текста"""
    
    df['text_sia'] = df['text'].apply(lambda x: sia_analysis.polarity_scores(x))
    df['text_neg'] = df['text_sia'].apply(lambda x: x['neg'])
    df['text_neu'] = df['text_sia'].apply(lambda x: x['neu'])
    df['text_pos'] = df['text_sia'].apply(lambda x: x['pos'])
    
    # главный параметр библиотеки SIA
    df['text_compound'] = df['text_sia'].apply(lambda x: x['compound'])
    
    return df


# применяем к нашим выборкам
train_df = sia_text_analysis(train_df)
test_df = sia_text_analysis(test_df)

А теперь для текста самого поста.

In [6]:
sia_analysis = SentimentIntensityAnalyzer()

def sia_post_analysis(df):
    """функция для получения различных коэффициентов семантической окраски текста"""
    
    df['post_sia'] = df['post'].apply(lambda x: sia_analysis.polarity_scores(x))
    df['post_neg'] = df['post_sia'].apply(lambda x: x['neg'])
    df['post_neu'] = df['post_sia'].apply(lambda x: x['neu'])
    df['post_pos'] = df['post_sia'].apply(lambda x: x['pos'])
    
    # главный параметр библиотеки SIA
    df['post_compound'] = df['post_sia'].apply(lambda x: x['compound'])
    df = df.drop('post_sia', axis=1)
    
    return df


# применяем к нашим выборкам
train_df = sia_post_analysis(train_df)
test_df = sia_post_analysis(test_df)

# AFINN

Аналогично сделаем анализ сначала для текста комментариев.

In [7]:
# вызываем afinn
afinn = Afinn()

def afinn_text_analysis(df):
    """функция для семантического анализа Afinn"""
    
    df['text_afinn_score'] = df['text'].apply(lambda x: afinn.score(x))
    
    return df


# применяем к нашим выборкам
train_df = afinn_text_analysis(train_df)
test_df = afinn_text_analysis(test_df)

А теперь для текста поста.

In [8]:
# вызываем afinn
afinn = Afinn()

def afinn_post_analysis(df):
    """функция для семантического анализа Afinn"""
    
    df['post_afinn_score'] = df['post'].apply(lambda x: afinn.score(x))
    
    return df


# применяем к нашим выборкам
train_df = afinn_post_analysis(train_df)
test_df = afinn_post_analysis(test_df)

# Text Blob

Узнаем полярность для текста комментариев.

In [9]:
def get_text_polarity(text):
    """функция для получения полярности комментария"""
    
    blob = TextBlob(text)
    
    return blob.sentiment_assessments.polarity


def text_blob_analysis(df):
    """функция для применения к выборкам"""
    
    df['text_blob_polarity'] = df['text'].apply(get_text_polarity)
    
    return df


# применяем к нашим выборкам
train_df = text_blob_analysis(train_df)
test_df = text_blob_analysis(test_df)

Узнаем полярность для текста поста.

In [10]:
def get_post_polarity(post):
    """функция для получения полярности комментария"""
    
    blob = TextBlob(post)
    
    return blob.sentiment_assessments.polarity


def post_blob_analysis(df):
    """функция для применения к выборкам"""
    
    df['post_blob_polarity'] = df['post'].apply(get_post_polarity)
    
    return df


# применяем к нашим выборкам
train_df = post_blob_analysis(train_df)
test_df = post_blob_analysis(test_df)

Узнаем позитивность комментария с помощью библиотеки TextBlob.

In [11]:
# вызываем другой метод обработки этой же библиотеки
text_blob = Blobber(analyzer=NaiveBayesAnalyzer())

def get_text_blobber_sentiment(text):
    """функция для получения позитивности текста - числовое значение"""
    
    blobber_sentiment = text_blob(text).sentiment
    
    return blobber_sentiment[1]


def get_text_blobber_positivity(text):
    """функция для проверка на позитивность - категориальное значение"""
    
    blobber_sentiment = text_blob(text).sentiment
    
    return blobber_sentiment[0]

def get_text_blob(df):
    
    df['text_blob_pos'] = df['text'].apply(get_text_blobber_sentiment)
    df['text_blob_neg'] = df['text'].apply(get_text_blobber_positivity)
    
    return df


train_df = get_text_blob(train_df)
test_df = get_text_blob(test_df)

Узнаем позитивность поста с помощью библиотеки TextBlob.

In [12]:
# вызываем другой метод обработки этой же библиотеки
post_blob = Blobber(analyzer=NaiveBayesAnalyzer())

def get_post_blobber_sentiment(post):
    """функция для получения позитивности текста - числовое значение"""
    
    blobber_sentiment = post_blob(post).sentiment
    
    return blobber_sentiment[1]


def get_post_blobber_positivity(post):
    """функция для проверка на позитивность - категориальное значение"""
    
    blobber_sentiment = post_blob(post).sentiment
    
    return blobber_sentiment[0]

def get_post_blob(df):
    
    df['post_blob_pos'] = df['post'].apply(get_post_blobber_sentiment)
    df['post_blob_neg'] = df['post'].apply(get_post_blobber_positivity)
    
    return df


train_df = get_post_blob(train_df)
test_df = get_post_blob(test_df)

Узнаем субъективность комментария.

In [13]:
# получаем параметр субъективности - комментарий основан на личном мнении или на фактах
text_blob = Blobber(analyzer=NaiveBayesAnalyzer())

def get_text_blobber_subjectivity(text):
    """функция для получения субъективности текста"""
    
    blobber_subject= text_blob(text).subjectivity
    
    return blobber_subject


def text_blob_subject(df):
    """функция для получения параметра субъективности текста"""
    
    df['text_blob_subject'] = df['text'].apply(get_text_blobber_subjectivity)
    
    return df

train_df = text_blob_subject(train_df)
test_df = text_blob_subject(test_df)

In [14]:
# категориальный признак позитивности текстовый, сразу его закодируем
def text_blob_is_positive(df):
    """функция для кодирования признака blob-позитивности"""
    
    df['text_blob_is_positive'] = df['text_blob_neg'].apply(lambda x: 1 if x=='pos' else 0)
    df = df.drop('text_blob_neg', axis=1)
    
    return df


# применяем к нашим выборкам
train_df = text_blob_is_positive(train_df)
test_df = text_blob_is_positive(test_df)

Узнаем субъективность поста.

In [15]:
# получаем параметр субъективности - пост основан на личном мнении или на фактах
post_blob = Blobber(analyzer=NaiveBayesAnalyzer())

def get_post_blobber_subjectivity(post):
    """функция для получения субъективности текста"""
    
    blobber_subject= post_blob(post).subjectivity
    
    return blobber_subject


def post_blob_subject(df):
    """функция для получения параметра субъективности текста"""
    
    df['post_blob_subject'] = df['post'].apply(get_post_blobber_subjectivity)
    
    return df

train_df = post_blob_subject(train_df)
test_df = post_blob_subject(test_df)

In [16]:
# категориальный признак позитивности текстовый, сразу его закодируем
def post_blob_is_positive(df):
    """функция для кодирования признака blob-позитивности"""
    
    df['post_blob_is_positive'] = df['post_blob_neg'].apply(lambda x: 1 if x=='pos' else 0)
    df = df.drop('post_blob_neg', axis=1)
    
    return df


# применяем к нашим выборкам
train_df = post_blob_is_positive(train_df)
test_df = post_blob_is_positive(test_df)

# Длина текста

Для комментария.

In [17]:
def get_text_length(df):
    """функция для получения длина комментария"""
    
    # разделяем комментарий на списки слов и считаем длину спика
    df['text_length'] = df['text'].str.split().str.len()
    
    return df

# применяем к нашим выборкам
train_df = get_text_length(train_df)
test_df = get_text_length(test_df)

Для поста.

In [18]:
def get_post_length(df):
    """функция для получения длина комментария"""
    
    # разделяем комментарий на списки слов и считаем длину спика
    df['post_length'] = df['post'].str.split().str.len()
    
    return df

# применяем к нашим выборкам
train_df = get_post_length(train_df)
test_df = get_post_length(test_df)

# Дополнительные синтетические признаки

In [21]:
def get_new_features(df):
    """ф-я для получения новых синтетических признаков - 
    разницы между различными параметрами поста и комментария"""
    
    # считаем разнице между семантическими показателями для поста и каждого комментария
    df['diff_compound'] = abs(df['post_compound'] - df['text_compound'])
    df['diff_afinn'] = abs(df['post_afinn_score'] - df['text_afinn_score'])
    df['diff_polarity'] = abs(df['post_blob_polarity'] - df['text_blob_polarity'])
    df['diff_blob_pos'] = abs(df['post_blob_pos'] - df['text_blob_pos'])
    df['diff_length'] = abs(df['post_length'] - df['text_length'])
    df['diff_subject'] = abs(df['post_blob_subject'] - df['text_blob_subject'])
    df['diff_is_positive'] = abs(df['post_blob_is_positive'] - df['text_blob_is_positive'])
    
    # перемножим разницы основных семантических библиотек с разницей с разницей в кол-ву слов
    df['sia_length'] = df['diff_compound'] * df['diff_length']
    df['afinn_length'] = df['diff_afinn'] * df['diff_length']
    df['blob_diff'] = df['diff_blob_pos'] * df['diff_length']
    
    return df


# применяем к нашим сэмплам
train_df = get_new_features(train_df)
test_df = get_new_features(test_df)

# АНАЛИЗ

Проверим очевидное - есть ли зависимость между рейтингом комментария и количеством слов в нем.

In [23]:
# сгруппируем данные для удобства
text_length_score_df = train_df.groupby('score')['text_length'].agg(['mean', 'median']).round().reset_index()

# строим график
text_score_length_plot = go.Figure()

# для среднего
text_score_length_plot.add_trace(go.Bar(
   x=text_length_score_df['score'],
   y=text_length_score_df['mean'],
   name='Среднее')
)

# для медианы
text_score_length_plot.add_trace(go.Bar(
   x=text_length_score_df['score'],
   y=text_length_score_df['median'],
   name='Медиана')
)

text_score_length_plot.update_layout(
   title='Зависимость рейтинга комментария от среднего и медианного количества слов',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Количество слов в комментарии',
   font=dict(size=16)
)

**Вывод:**  
Очевидно, зависимость есть и она прямо пропорциональна. С уменьшением количества слов (в среднем или по медиане) снижается и рейтинг комментария. При этом самыми популярными, судя по этому графику, становятся комментарии, где в среднем около 139 слов, т.е. довольно длинная цепь рассуждений/повествования.
___________

Теперь оценим зависимость рейтинга комментария от его семантической окраски с учетом библиотеки.

В библиотеке Sentiment Intensity Analyzer ключевая метрика называется compound. Она может принимать отрицательные (для негативных комментариев) и положительные (для позитивных) значения. Для более полной картины, чтобы не перемешивать (+) и (-) значения и не получить ерунду, будет рассматривать их по отдельности.

In [33]:
# отделяем отрицательную оценку и группируем по рейтингу
neg_mask = train_df[train_df['text_compound'] < 0]
sia_neg_df = neg_mask.groupby('score')['text_compound'].agg(['mean', 'median']).round(2).reset_index()

In [34]:
# отделяем неотрицательные оценки и группируем по рейтингу
pos_mask = train_df[train_df['text_compound'] >= 0]
sia_pos_df = pos_mask.groupby('score')['text_compound'].agg(['mean', 'median']).round(2).reset_index()

In [35]:
sia_plot = go.Figure()

# для среднего
sia_plot.add_trace(go.Bar(
   x=sia_neg_df['score'],
   y=sia_neg_df['mean'],
   name='Среднее для негативных')
)

sia_plot.add_trace(go.Bar(
   x=sia_neg_df['score'],
   y=sia_neg_df['median'],
   name='Медиана для негативных')
)

# для медианы
sia_plot.add_trace(go.Bar(
   x=sia_pos_df['score'],
   y=sia_pos_df['mean'],
   name='Среднее для позитивных')
)
sia_plot.add_trace(go.Bar(
   x=sia_pos_df['score'],
   y=sia_pos_df['median'],
   name='Медиана для позитивных')
)

sia_plot.update_layout(
   title='Зависимость рейтинга комментария от семантического анализа текста (библиотека SIA)',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Оценка Sentiment Intensity Analyzer'
)

**Выводы:**  
Изначально мной был сделан неверный вывод, поэтому я и разделил график на + и - (по мнению библиотеки SIA) комментарии. Чем больше значение получил комментарий по модулю, тем более он экспресивен и эмоционален. И по графику мы видим очень четкую прямую зависимость - чем более яркий в семантическом плане комментарий, тем выше его рейтинг. При этом независимо от его позитивности или негативности.
_________

Теперь посмотрим есть ли зависимость между рейтингом и семантическим анализом поста и комментария для библиотеки SIA. Аналогично, для наглядности будем брать только средние значения.

In [36]:
# для комментария
# отделяем отрицательную оценку и группируем по рейтингу
text_neg = train_df[train_df['text_compound'] < 0]
text_sia_neg_df = text_neg.groupby('score')['text_compound'].mean().round(2).reset_index()

# отделяем неотрицательные оценки и группируем по рейтингу
text_pos = train_df[train_df['text_compound'] >= 0]
text_sia_pos_df = text_pos.groupby('score')['text_compound'].mean().round(2).reset_index()

In [37]:
# теперь для поста
# отделяем отрицательную оценку и группируем по рейтингу
post_neg = train_df[train_df['post_compound'] < 0]
post_sia_neg_df = post_neg.groupby('score')['post_compound'].mean().round(2).reset_index()

# отделяем неотрицательные оценки и группируем по рейтингу
post_pos = train_df[train_df['post_compound'] >= 0]
post_sia_pos_df = post_pos.groupby('score')['post_compound'].mean().round(2).reset_index()

In [41]:
text_post_sia = go.Figure()

# - для комментария
text_post_sia.add_trace(go.Bar(
   x=text_sia_neg_df['score'],
   y=text_sia_neg_df['text_compound'],
   name='Отрицательный комментарий')
)
# - для поста
text_post_sia.add_trace(go.Bar(
   x=post_sia_neg_df['score'],
   y=post_sia_neg_df['post_compound'],
   name='Отрицательный пост')
)

# + для комментария
text_post_sia.add_trace(go.Bar(
   x=text_sia_pos_df['score'],
   y=text_sia_pos_df['text_compound'],
   name='Положительный комментарий')
)

# + для поста
text_post_sia.add_trace(go.Bar(
   x=post_sia_pos_df['score'],
   y=post_sia_pos_df['post_compound'],
   name='Положительный пост')
)


text_post_sia.update_layout(
   title='Оценка SIA для комментария и поста в зависимости от рейтинга',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Оценка Sentiment Intensity Analyzer'
)

**Выводы:**  
В этом графике мы дополнили семантическую оценку комментария семантической оценкой поста. Сделанный ранее вывод можно смело дополнить: и для негативных постов, и для позитивных более высокая экспрессия комментария способствует его высокой значимости.
__________

Посмотрим работает ли это правило для библиотеки Afinn.

In [43]:
# отделяем отрицательную оценку и группируем по рейтингу
neg_mask = train_df[train_df['text_afinn_score'] < 0]
afinn_neg_df = neg_mask.groupby('score')['text_afinn_score'].agg(['mean', 'median']).round(2).reset_index()

In [44]:
# отделяем неотрицательные оценки и группируем по рейтингу
pos_mask = train_df[train_df['text_afinn_score'] >= 0]
afinn_pos_df = pos_mask.groupby('score')['text_afinn_score'].agg(['mean', 'median']).round(2).reset_index()

In [45]:
# сравнивать для наглядности буем также медианное и среднее значение
afinn_plot = go.Figure()

# для - среднего
afinn_plot.add_trace(go.Bar(
   x=afinn_neg_df['score'],
   y=afinn_neg_df['mean'],
   name='Среднее для негативных')
)

# для - медианы
afinn_plot.add_trace(go.Bar(
   x=afinn_neg_df['score'],
   y=afinn_neg_df['median'],
   name='Медиана для негативных')
)

# для + среднего
afinn_plot.add_trace(go.Bar(
   x=afinn_pos_df['score'],
   y=afinn_pos_df['mean'],
   name='Среднее для позитивных')
)

# для - среднего
afinn_plot.add_trace(go.Bar(
   x=afinn_pos_df['score'],
   y=afinn_pos_df['median'],
   name='Медиана для позитивных')
)

# добавляем подписи
afinn_plot.update_layout(
   title='Зависимость рейтинга комментария от семантического анализа текста (библиотека Afinn)',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Оценка Afinn'
)

**Выводы:**  
Библиотека Afinn только подтвержает сделанный ранее вывод. При этом на этом графике картина даже более наглядная.
___________

Но, на всякий случай, посмотрим результаты библиотеки Blob.

In [47]:
# отделяем отрицательную оценку и группируем по рейтингу
neg_mask = train_df[train_df['text_blob_polarity'] < 0]
blob_neg_df = neg_mask.groupby('score')['text_blob_polarity'].agg(['mean', 'median']).round(2).reset_index()

In [48]:
# отделяем неотрицательные оценки и группируем по рейтингу
pos_mask = train_df[train_df['text_blob_polarity'] >= 0]
blob_pos_df = pos_mask.groupby('score')['text_blob_polarity'].agg(['mean', 'median']).round(2).reset_index()

In [49]:
# сравнивать для наглядности будем также медианное и среднее значение
blob_plot = go.Figure()

# для - среднего
blob_plot.add_trace(go.Bar(
   x=blob_neg_df['score'],
   y=blob_neg_df['mean'],
   name='Среднее для негативных')
)

# для - медианы
blob_plot.add_trace(go.Bar(
   x=blob_neg_df['score'],
   y=blob_neg_df['median'],
   name='Медиана для негативных')
)

# для + среднего
blob_plot.add_trace(go.Bar(
   x=blob_pos_df['score'],
   y=blob_pos_df['mean'],
   name='Среднее для позитивных')
)

# для - среднего
blob_plot.add_trace(go.Bar(
   x=blob_pos_df['score'],
   y=blob_pos_df['median'],
   name='Медиана для позитивных')
)

# добавляем подписи
blob_plot.update_layout(
   title='Зависимость рейтинга комментария от семантического анализа текста (библиотека Blob)',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Оценка Blob'
)

**Выводы:**  
Здесь зависимость обратная, однако это связанно с особенностями анализа от библиотеки TextBlob.
___________

Теперь оценим влияние фактора субъективности текста.

In [51]:
# сгруппируем данные для удобства
subjectivity_score_df = train_df.groupby('score')['text_blob_subject'].agg(['mean', 'median']).round(4).reset_index()

# строим график
subjectivity_score_plot = go.Figure()

# для среднего
subjectivity_score_plot.add_trace(go.Bar(
   x=subjectivity_score_df['score'],
   y=subjectivity_score_df['mean'],
   name='Среднее')
)

# для медианы
subjectivity_score_plot.add_trace(go.Bar(
   x=subjectivity_score_df['score'],
   y=subjectivity_score_df['median'],
   name='Медиана')
)

subjectivity_score_plot.update_layout(
   title='Зависимость рейтинга комментария от среднего и медианного значения субъективности',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Субъективность комментария'
)

**Выводы:**  
Зависимость почти незаметная, скорее ее и вовсе нет, но сложно сказать без дополнительной оценки статистической значимости. Однако в сотых и тысячных значениях субъективность в более релевантных комментариях выше, т.е. основаных больше на личной информации, чем на фактах.
___________

Попробуем теперь оценить разницу между семантической оценкой поста и комментария в зависимости от рейтинга комментарияю.

In [79]:
diff_compound_df = train_df.groupby('score')['diff_compound'].agg(['mean', 'median']).reset_index().round(2)

# строим график
diff_compound_plot = go.Figure()

# для среднего
diff_compound_plot.add_trace(go.Bar(
   x=diff_compound_df['score'],
   y=diff_compound_df['mean'],
   name='Среднее')
)

# для медианы
diff_compound_plot.add_trace(go.Bar(
   x=diff_compound_df['score'],
   y=diff_compound_df['median'],
   name='Медиана')
)

diff_compound_plot.update_layout(
   title='Разница эмоциональной окраски поста и комментария в разрезе оценки',
   xaxis_title='Рейтинг комментария',
   yaxis_title='Оценка Sentiment Intensity Analyzer'
)

**Выводы:**  
Кажется, что такой график мы уже видели, а вывод очевиден. Однако в этот раз мы оценивали разницу в оценке. Теперь мы понимаем, что чем большая разница в эмоциональной окраске комментария к посту, тем выше значимость такого комментария. Таким образом, можно подумать, что высказывание противоположной позиции (позитива на негатив или наоборот) имеет более высокие шансы оказаться в топе комментариев. Проверим это.
_____________

Ранее мы получили признак diff_is_positive - признак, созданный с помощью библиотеки TextBlob, который показывает отличается ли позитивность поста и комментария:
- если 0 - значит пост и комментарий в одной эмоциональной окраске, т.е. оба негативные или оба позитивные
- если 1 - значит их окраска противоположна.
Соответственно, чем больше значение в каждой группе рейтинга комментария, тем больше в этом рейтинге противоположностей.

In [125]:
diff_positive_df = train_df.groupby('score')['diff_is_positive'].sum().reset_index()

diff_positive_plot = px.bar(
    data_frame=diff_positive_df,
    x='score',
    y='diff_is_positive',
    color='diff_is_positive',
    title='Число противоположных окрасок между постом и комментарием в разрезе рейтинга'
)

diff_positive_plot.update_layout(
    xaxis_title='Рейтинг комментария',
    yaxis_title='Количество отличий',
    title=dict(x=.5, xanchor='center')
)

diff_positive_plot.show()

**Выводы:**  
А теперь мы видим, что негатив на позитив (или наоборот) более вероятно попадет в категорию важных комментариев. Таким образом, эмоциональные и экспрессивные комментарии имеют больший рейтинг, но в это же время не должны иметь противоположный посту эмоциональный оттенок.
___________